# Gridding and PICS Reconstruction for Philips UTE Koosh Acquisition
* Gridding / NuFFT
    * mathematically derived density compensation factor
    * sigpy dcf
* JSENSE 
* PICS
    * Ecalib & PICS

In [ ]:
import numpy as np
%matplotlib qt
import sigpy.plot as pl
import sigpy as sp

In [ ]:
# load data & trajectory saved by gpilab
file_dir = '/path_to_file/prefix_'
data_dir = file_dir + 'data.npy'
traj_dir = file_dir + 'traj.npy'
data = np.load(data_dir)
traj = np.load(traj_dir)

In [ ]:
# scale fov differently in each direction 
fov_scale = np.array([0.7, 0.7, 1.3]) # philips orientation [S/I, A/P, L/R]
fov_scale = fov_scale[..., np.newaxis, np.newaxis]
traj_scale = traj * fov_scale

In [ ]:
# transpose data & traj
data_t = np.squeeze(data.T)
traj_scale_t = np.real(traj_scale.T)

NuFFT Gridding Recon

In [ ]:
# sigpy dcf
import sigpy.mri as mr
dcf2 = mr.dcf.pipe_menon_dcf(traj_scale_t)

In [ ]:
# gridding
recon_array2 = [sp.nufft_adjoint(data_t[n,:,:] * dcf2, traj_scale_t) for n in range(data_t.shape[0]) ]
recon2 = np.stack(recon_array2, axis=0) 
recon_sos2 = np.sum(np.abs(recon2)**2, axis=0)**0.5 # Sum of Square
np.save(file_dir + "recon_sos2.npy", recon_sos2)
pl.ImagePlot(recon_sos2, title='NuFFT Reconstruction', x = -1, y = -3)

JSENSE Recon

In [ ]:
# maps estimation
ksp = sp.fft(recon2, axes = [1,2,3])
mps = mr.app.JsenseRecon(ksp).run()

In [ ]:
lamda = 0.01
img_sense = mr.app.SenseRecon(ksp, mps, lamda=lamda).run()
np.save(file_dir + "recon_sense.npy", img_sense)
pl.ImagePlot(img_sense, title='SENSE Reconstruction', x = -1, y = -3)

PICS l1 wavelet regularized Recon

In [ ]:
lamda = 0.005
img_l1wav = mr.app.L1WaveletRecon(ksp, mps, lamda).run()
np.save(file_dir + "recon_l1wav.npy", img_l1wav)
pl.ImagePlot(img_l1wav, title='PICS L1 Wavelet Regularized Reconstruction', x = -1, y = -3)